RAG of UAE data with Falcon

In [ ]:
!pip install -U -q "langchain" "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Data Parsing

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

In [ ]:
#connect to gdrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
dir = "/content/drive/MyDrive/Projects/Omdena_UAE_Falcon/data/"

faculty_member = CSVLoader(file_path=dir + "faculty_member_data.csv",source_column="year")
government_education = CSVLoader(file_path=dir + "government_education_data.csv",source_column="Year")
government_private_education = CSVLoader(file_path=dir + "government_private_education_data.csv",source_column="year")
library = CSVLoader(file_path=dir + "library_data.csv",source_column="year")
scholarship = CSVLoader(file_path=dir + "scholarship_data.csv",source_column="Degree_en")

In [ ]:
faculty_member_data = faculty_member.load()
government_education_data = government_education.load()
government_private_education_data = government_private_education.load()
library_data = library.load()
scholarship_data = scholarship.load()

In [ ]:
import pandas as pd

faculty_df = pd.read_csv(dir + "faculty_member_data.csv")
faculty_df.head()

,year,Title_en,Gender_en,value,year_2014 / 2013,year_2015 / 2014,year_2016 / 2015,year_2017 / 2016,Gender_en_Female,Gender_en_Male,Gender_en_Mix,Gender_en_…,Title_en_Schools,Title_en_Students,"Title_en_Teaching, staff",Title_en_classrooms
0,2014 / 2013,Schools,Male,259,1,0,0,0,0,1,0,0,1,0,0,0
1,2014 / 2013,"Teaching, staff",Male,6955,1,0,0,0,0,1,0,0,0,0,1,0
2,2014 / 2013,Students,Male,114167,1,0,0,0,0,1,0,0,0,1,0,0
3,2014 / 2013,Schools,Female,236,1,0,0,0,1,0,0,0,1,0,0,0
4,2014 / 2013,"Teaching, staff",Female,17960,1,0,0,0,1,0,0,0,0,0,1,0


In [ ]:
len(faculty_member_data)

32

In [ ]:
len(government_education_data)

64

In [ ]:
len(government_private_education_data)

32

In [ ]:
len(library_data)

69

In [ ]:
len(scholarship_data)

72

### Text Splitter

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap = 5,
    length_function = len,
)

In [ ]:
library_doc = text_splitter.transform_documents(library_data)

In [ ]:
len(library_doc)

96

### Create Index

In [ ]:
!pip install -q -U faiss-cpu tiktoken sentence-transformers

In [ ]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS    #vector store
from langchain.storage import LocalFileStore
from torch import cuda

store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

#embedding cache to save time if same questions are asked.
embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.from_documents(library_doc, embedder)

In [ ]:
#check if the vectorstore is working correctly.
query = "What are the subjects of books present in public libraries in UAE?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k=4)

for page in docs:
  print(page.page_content)

year: 2014
title_en: Humanities and Arts
value: 43
title_en: Appiled Sciences & Pure Sciences
title_en: Appiled Sciences & Pure Sciences
title_en: Appiled Sciences & Pure Sciences


### Build a Retrieval Chain

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install huggingface-hub -q

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.4 MB/s eta 0:00:00


In [ ]:
import torch
import transformers

model_id = 'tiiuae/falcon-7b-instruct'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code = True,
    config=model_config,
    quantization_config=bnb_config,
    device_map="auto",
)

model.eval()

Loading tiiuae/falcon-7b-instruct requires to execute some code in that repo, you can inspect the content of the repository at https://hf.co/tiiuae/falcon-7b-instruct. You can dismiss this prompt by passing `trust_remote_code=True`.
Do you accept? [y/N] y


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RWForCausalLM(
  (transformer): RWModel(
    (word_embeddings): Embedding(65024, 4544)
    (h): ModuleList(
      (0-31): 32 x DecoderLayer(
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
        (self_attention): Attention(
          (maybe_rotary): RotaryEmbedding()
          (query_key_value): Linear4bit(in_features=4544, out_features=4672, bias=False)
          (dense): Linear4bit(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): MLP(
          (dense_h_to_4h): Linear4bit(in_features=4544, out_features=18176, bias=False)
          (act): GELU(approximate='none')
          (dense_4h_to_h): Linear4bit(in_features=18176, out_features=4544, bias=False)
        )
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4544, out_features=65024, bias=False)
)

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

In [ ]:
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 8.4 MB/s eta 0:00:00


In [ ]:
#create pipeline
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    return_full_text=True,
    temperature=0.0,
    max_new_tokens=256
)

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

retrieval_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

In [ ]:
retrieval_chain({"query":"What are the subjects of books present in public libraries in the UAE?"})

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What are the subjects of books present in public libraries in the UAE?',
 'result': '\n\nThe subjects of books present in public libraries in the UAE include Humanities, Arts, Applied Sciences, Pure Sciences, and more.',
 'source_documents': [Document(page_content='year: 2014\ntitle_en: Humanities and Arts\nvalue: 43', metadata={'source': '2014', 'row': 13}),
  Document(page_content='title_en: Appiled Sciences & Pure Sciences', metadata={'source': '2014', 'row': 7}),
  Document(page_content='title_en: Appiled Sciences & Pure Sciences', metadata={'source': '2015', 'row': 30}),
  Document(page_content='title_en: Appiled Sciences & Pure Sciences', metadata={'source': '2016', 'row': 53})]}

In [ ]:
retrieval_chain({"query":"You are a data scientist. Can you generate a python code that plots a bar graph to display the count of Philosophy and Psychology books over the years?"})

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'You are a data scientist. Can you generate a python code that plots a bar graph to display the count of Philosophy and Psychology books over the years?',
 'result': "\n\nimport matplotlib.pyplot as plt\n\ndata = {'year': ['2014', '2016', '2015'], 'title_en': ['Philosophy and psychology', 'Philosophy and psychology', 'Social Sciences'], 'count': [127795, 128000, 127000]}\n\nplt.bar(data['year'], data['count'], label=data['title_en'])\nplt.xlabel('Year')\nplt.ylabel('Count')\nplt.show()",
 'source_documents': [Document(page_content='year: 2014\ntitle_en: Philosophy and psychology', metadata={'source': '2014', 'row': 1}),
  Document(page_content='year: 2016\ntitle_en: Philosophy and psychology', metadata={'source': '2016', 'row': 47}),
  Document(page_content='year: 2015\ntitle_en: Philosophy and psychology', metadata={'source': '2015', 'row': 24}),
  Document(page_content='year: 2014\ntitle_en: Social Sciences\nvalue: 127795', metadata={'source': '2014', 'row': 3})]}

In [ ]:
retrieval_chain({"query":"You are a data scientist. Plot a bar graph to display the count of Philosophy and Psychology books over the years."})

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'You are a data scientist. Plot a bar graph to display the count of Philosophy and Psychology books over the years.',
 'result': "\n\nThe following code creates a bar graph to display the count of Philosophy and Psychology books over the years:\n\n```\nimport matplotlib.pyplot as plt\n\ndata = {'year': ['2014', '2016', '2015'], 'title_en': ['Philosophy and psychology', 'Philosophy and psychology', 'Social Sciences']}\n\nplt.bar(data['year'], data['title_en'], width=0.5)\nplt.xlabel('Year')\nplt.ylabel('Number of Books')\nplt.show()\n```\n\nOutput:\n\n![Philosophy and Psychology Bar Graph](https://i.imgur.com/KzKwZf.png)",
 'source_documents': [Document(page_content='year: 2014\ntitle_en: Philosophy and psychology', metadata={'source': '2014', 'row': 1}),
  Document(page_content='year: 2016\ntitle_en: Philosophy and psychology', metadata={'source': '2016', 'row': 47}),
  Document(page_content='year: 2015\ntitle_en: Philosophy and psychology', metadata={'source': '2015', 'row':

### Scholarship dataset

In [ ]:
scholarship_doc = text_splitter.transform_documents(scholarship_data)

In [ ]:
len(scholarship_doc)

522

In [ ]:
vector_store = FAISS.from_documents(scholarship_doc, embedder)

In [ ]:
#check if the vectorstore is working correctly.
query = "Identify the degree level and gender with the highest and lowest number of scholarships and graduates."
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k=3)

for page in docs:
  print(page.page_content)

Studenton_en: Scholarships
Gender_en: Female
Studenton_en: Scholarships
Gender_en: Female
Studenton_en: Scholarships
Gender_en: Female


In [ ]:
retrieval_chain({"query":"Identify the degree level and gender with the highest and lowest number of scholarships and graduates."})

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Identify the degree level and gender with the highest and lowest number of scholarships and graduates.',
 'result': '\n\nYear | Title_en | Value\n--- | --- | ---\n2016 | Education | 63\n2015 | Education | 206\n2014 | Humanities and Arts | 58\n\nThe degree level with the highest number of scholarships and graduates is Education, with 63 scholarships and 206 graduates. The degree level with the lowest number of scholarships and graduates is Humanities and Arts, with 25 scholarships and 25 graduates.',
 'source_documents': [Document(page_content='year: 2016\ntitle_en: Education\nvalue: 63', metadata={'source': '2016', 'row': 58}),
  Document(page_content='year: 2015\ntitle_en: Education\nvalue: 206', metadata={'source': '2015', 'row': 35}),
  Document(page_content='year: 2014\ntitle_en: Education\nvalue: 58', metadata={'source': '2014', 'row': 12}),
  Document(page_content='year: 2016\ntitle_en: Humanities and Arts\nvalue: 25', metadata={'source': '2016', 'row': 59})]}

In [ ]:
retrieval_chain({"query":"Calculate the percentage of scholarships and graduates for each degree level and gender."})

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Calculate the percentage of scholarships and graduates for each degree level and gender.',
 'result': '\n\nFor the year 2016, there were 63 scholarships for the Education degree level, which is 100% of the total. For the same degree level, there were 206 graduates, which is 100% of the total. For the Humanities and Arts degree level, there were 25 scholarships, which is 100% of the total. For the same degree level, there were 58 graduates, which is 100% of the total.',
 'source_documents': [Document(page_content='year: 2016\ntitle_en: Education\nvalue: 63', metadata={'source': '2016', 'row': 58}),
  Document(page_content='year: 2015\ntitle_en: Education\nvalue: 206', metadata={'source': '2015', 'row': 35}),
  Document(page_content='year: 2014\ntitle_en: Education\nvalue: 58', metadata={'source': '2014', 'row': 12}),
  Document(page_content='year: 2016\ntitle_en: Humanities and Arts\nvalue: 25', metadata={'source': '2016', 'row': 59})]}